# Predicting the Source of Wildfires
Luke Waninger

DATA 512 Final Project

University of Washington, Fall 2018

## Introduction

Wildfires have been a big topic in the recent news with devasting effects across the western coast of the United States. So far this year, we have had less burn than 2017, but the current fire in California is the largest in state history and still burns rapidly. Last year, we had almost 2 billion dollars of losses across the United States as a result of wildfire damage which has been the highest in history. Risks of wildfires continue to climb as scientists discover alarming links between rising greenhouse gasses, temperature, and wildfire severity. N. P. Gillett et al. performed a comprehensive study on the relationship between the two and concluded with overwhelming confidence that a positive trend exists between them. Rising greenhouse gasses could be playing a significant role in the prevalence and severity of forest fires.

Key to understanding the overall problem is the double-edged sword forests play in climate change; they are both a cause and effect. The wildfires both increase atmospheric greenhouse gasses and destroy the integral vegetation to the planet's carbon cycle. The Paris Agreement has specifically mentioned the importance of this and insists that countries protect against deforestation. Not only is the world pushing to keep the forests we have but here at home, we have begun to employ them as significant combatants in the fight against climate change. California has led the way with their proposed carbon plan. It proposes methods to reshape parts of their existing ecosystem to make their forests even more efficient at removing carbon. Stopping deforestation would significantly promote the UNs progress towards reaching goals outlined in the Paris Agreement.

However, this will not work if the forests continue in the same destructive cycle with our ecosystem. The goal of this project is two-fold. One, to understand the independent variables and correlation effects in a combined dataset of the Fire Program Analysis (FPA) reporting system, NOAA's Global Surface Summary of Day Data (GSOD) 7, and  NASA's biomass indicators. Two, to train and assess a model for predicting the reason a wildfire started. (and possibly estimate the impact? location?) Identifying the source is a difficult task for investigators in the wild. The vastness of land covered is much larger than the matchstick or location of a lightning strike. Developing an understanding of the independent variables and a reliable prediction model could give authorities valuable direction as to where to begin their search.

#### Research Questions
* What are the most important indicators to consider when determining the strength of a wildfire?
* Can a reliable model be built to assist investigators in determining the cause of a wildfire?

####  Reproducibility
This notebook is intended to be completely reproducible. However, the final dataset is much too large to be hosted on GitHub. I provide a small, randomly selected sample with the repository to show the dataset cleaning and generation process. The full dataset can be downloaded by running the following cell. Running it will overwrite the sample provided. To undoe this effect, reclone the repository. I highly advise not to attempting to run the full dataset. Generating the weather aggregations across the entire fires dataset can take a considerable amount of time. With 12 cores running at 4ghz and a consistent 95% CPU load, it took my machine nearly 27 hours to compute.

Additionally, this notebook is run as two major parts. Part 1 consists of data cleaning and final dataset creation. This can be run any machine with at least 20gb of memory. The seconds portion is analysis which requires a much larger system to compute. Full directions are shown below.

## Part 1. Data Preparation

### Setup
This notebook is coded to run with Python 3.6. Several libraries from the Python standard library will be used along with several third-party modules. These can be installed with the provided requirements file using the command 

`pip install --user -r requirements.txt`

More information regarding the standard libarary can be found at [python.org](https://docs.python.org/3.6/library/index.html).

For third party libraries please see:
* [numpy == 1.13.0](https://docs.scipy.org/doc/numpy-1.13.0/reference/)
* [pandas == 0.23.4](https://pandas.pydata.org/pandas-docs/stable/)
* [plotly == 3.4.2](https://plot.ly/python/)
* [statsmodels == 0.9.0](https://www.statsmodels.org/stable/index.html)
* [tqdm == 4.28.1](https://github.com/tqdm/tqdm)

In [12]:
# Python standard library
import datetime as dt
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame
import itertools as it
import multiprocessing as mul
from multiprocessing.dummy import Pool as TPool
import os
import gzip
import sqlite3
import sys
import tarfile
import zipfile

# third party modules
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tqdm import tqdm, tqdm_pandas

# initialize plotly    
#init_notebook_mode(connected=True)


# set notebook options
InteractiveShell.ast_node_interactivity = 'all'

# initalize tqdm
tqdm.pandas(leave=True)

### Data Sources
Four data sources are to be used for this project. The primary data source was found through Kaggle and contains 1.88 million wildfires that occurred in the United States from 1992 to 2015. This data contains the primary labels to be used as target variables. The United States Department of Agriculture curated the original data ([Forest Service](https://www.fs.fed.us/)) and can be found at [link](https://www.fs.usda.gov/rds/archive/Product/RDS-2013-0009.4/). The second is the GSOD data curated by [NOAA](https://www.noaa.gov/). Finally, the National Air and Space Association (NASA) hosts a valuable biome dataset at the ORNL Distributed Active Archive Center for Biogeochemical Dynamics ([DAAC](https://daac.ornl.gov/NPP/guides/NPP_EMDI.html). Later in the notebook, I will show how neither the NASA or DAAC data is useful and propose an alternate data source for future work.

### Get some metrics from the fires dataset
The target variable for this analysis exists inside the wildfire dataset. I start by generating a bounding box of latitude and longitude values to filter the other three sources.

In [ ]:
# generate the file path and connect using the sqlite3 driver
path = os.path.join('.', 'data', 'FPA_FOD_20170508.sqlite')
conn  = sqlite3.connect(path)

# retrieving the minimum and maximum latitude and longitude pairs.
fires = pd.read_sql_query('''
    SELECT 
        min(LATITUDE)  AS min_lat,
        max(LATITUDE)  AS max_lat,
        min(LONGITUDE) AS min_lon,
        max(LONGITUDE) AS max_lon
    FROM
        Fires
''', conn)

# increase by one degree-decimal point so that we don't exclude 
# nearby weather stations
min_lat = np.round(fires.min_lat.values[0], 0)-1
min_lon = np.round(fires.min_lon.values[0], 0)-1
max_lat = np.round(fires.max_lat.values[0], 0)+1
max_lon = np.round(fires.max_lon.values[0], 0)+1

# print them to the console
min_lat, max_lat, min_lon, max_lon

### Load and process GSOD files
The data from NOAA comes in separate zip files, one compressed tar file for each year. Then, each day of the year is yet another compressed gzip file. I extract the main file and remove any years not from 1991-2015. In the next cell I unzip the years we need, then each year into the directory 'gsod_extracted'.

In [ ]:
# create the file path
gsod_path = os.path.join('.', 'data', 'gsod')

# make sure the path exists
if not os.path.exists(gsod_path):
    os.mkdir(gsod_path, parents=True)
    
# get the main zip file
all_years = zipfile.ZipFile(os.path.join('data','gsod_all_years.zip'))

# look for contents only in the designated year range
members   = [
    n for n in all_years.namelist() 
    if any([n.find(str(yi)) > -1 for yi in list(range(1991, 2016))])
]

# extract
for m in tqdm(members):
    all_years.extract(m, gsod_path)

For each year, extract the daily station files.

In [ ]:
# get the yearly list of tar files
years = [f for f in os.listdir(gsod_path) if f.find('tar') > -1]

# generate the extract path
ex_path = os.path.join('.', 'data', 'gsod_extracted')

# make sure the path exists
if not os.path.exists(ex_path):
    os.mkdir(ex_path, parents=True)
    
# extract the content from each year into the 'extracted' directory
pbar = tqdm(total=len(years))
for y in years:
    pbar.set_description(y)
    
    # load the tarfile provided by NOAA
    tf = tarfile.TarFile(os.path.join(gsod_path, y))
    
    # create a subdirectory to extract the contents into
    subdir = os.path.join(ex_path, y.replace('.tar', ''))
    if not os.path.exists(subdir):
        os.mkdir(subdir)
    
    # extract each year
    tf.extractall(subdir)    
    pbar.update(1)

Process each station file line-by-line into DataFrame. This cell only does the raw transformation from a gzip text file into a csv. Each line of each file is a separate row with each field separated by a certain number of character positions. These are listed in the NOAA GSOD docs and were extensively used to process the data. Note, the extractions do not line up perfectly due to the parser being used. Each column was carefully checked to ensure no missing characters. Also of note is that some of the files contain blank lines so I added a filter at the end of each parsing to only input the row if a valid station id is present. We can't perform the latitude, longitude lookup without it making the row unusable even if it did contain the remaining fields.

In [ ]:
# get the list of years to process
years = [f for f in os.listdir(gsod_path) if f.find('tar') > -1]

# get the list of files for each day
ex_path = os.path.join('.', 'data', 'gsod_extracted')

# read and extract the contents for each day of year
i=0
for y in years:
    # create the filename to save the final csv output
    name = os.path.join(ex_path, y.replace('.tar', '.csv'))
    
    # get the subdirectory path
    subdir = os.path.join(ex_path, y.replace('.tar', ''))
    
    # read all files we extracted into the directory
    files = os.listdir(subdir)
    
    # store a list of dictionary objects for each row parsed
    content = []

    for f in tqdm(files, desc=y):
        # open the file
        with gzip.open(os.path.join(subdir, f), 'r') as fc:
            # read the entire contents, split by newline and ignore header
            t = str(fc.read()).split('\\n')[1:]

            # see GSOD_DESC.txt for exact delimmiter locations
            def parse(s):
                d = dict(
                    stn      = s[ 0: 6].strip(),
                    wban     = s[ 6:13].strip(),
                    year     = s[13:18].strip(),
                    moda     = s[18:23].strip(),
                    temp     = s[23:30].strip(),
                    temp_cnt = s[30:34].strip(),
                    dewp     = s[34:41].strip(),
                    dewp_cnt = s[41:44].strip(),
                    slp      = s[44:52].strip(),
                    slp_cnt  = s[52:55].strip(),
                    stp      = s[55:63].strip(),
                    stp_cnt  = s[63:66].strip(),
                    visib    = s[67:73].strip(),
                    visib_cnt= s[73:76].strip(),
                    wdsp     = s[76:83].strip(),
                    wdsp_cnt = s[83:86].strip(),
                    mxspd    = s[88:93].strip(),
                    gust     = s[94:101].strip(),
                    temp_max = s[102:108].strip(),
                    max_temp_flag = s[108:110].strip(),
                    temp_min = s[111:116].strip(),
                    min_temp_flag = s[116:117].strip(),
                    prcp     = s[117:123].strip(),
                    prcp_flag= s[123:124].strip(),
                    sndp     = s[124:131].strip(),
                    frshtt   = s[131:138].strip()
                )
                
                return d if len(d['stn']) > 1 else None

            # convert each row into a dictionary using the function above
            # and append the contents to the main collection
            content += list(map(parse, t))
    
    # convert the list of dictionaries to a Pandas dataframe
    content = pd.DataFrame([c for c in content if c is not None])
    
    # write this years worth of weather recordings to csv
    content.to_csv(name, index=None)                                

Now that we have the data in a usable format, we need join with the weather stations listing in order to gather the latitude and longitude values for each weather summary. I do this by creating a composite key out of USAF and WBAN in both the stations and weather dataframes, then performing an inner join on it. For more information please see the NOAA data documentation provided.

I also make sure to exclude weather stations that aren't going to be used in widlfire feature engineering by creating latitude and longitude masks offsetting each min/max by 111km.

In [ ]:
# load the stations file explicitly enforcing datatypes and nan values
# also drop any station that doesn't have a latitude or longitude value
stations = pd.read_csv(
    os.path.join('data', 'isd-history.csv'),
    dtype={
        'USAF':'str',
        'WBAN':'str'
    },
    na_values={
        'WBAN'   :'99999',
        'ELEV(M)':'-999'
    }
).dropna(subset=['LAT', 'LON'], how='any')

# take only stations that have lat, lon values within the wildfire range
stations['lat_mask'] = [min_lat <= lat <= max_lat for lat in stations.LAT]
stations['lon_mask'] = [min_lon <= lon <= max_lon for lon in stations.LON]
stations = stations.loc[stations.lat_mask & stations.lon_mask].drop(columns=['lat_mask', 'lon_mask'])

# create a key by concatenating the USAF and WBAN cols
stations.loc[stations.USAF.isnull(), 'USAF'] = 'none'
stations.loc[stations.WBAN.isnull(), 'WBAN'] = 'none'
stations['KEY'] = stations.USAF+stations.WBAN

# verify key uniqueness
assert len(stations.KEY.unique()) == len(stations)

# we will only be using these columns
stations = stations.reindex(columns=[
    'KEY', 'LAT', 'LON', 'ELEV(M)'
])

# rename the elevation column so we can call it easier
stations = stations.rename(columns={'ELEV(M)':'ELEV'})

# convert all the column names to lowercase
stations.columns = [c.lower() for c in stations.columns]

stations.head()

In the following cell, I go through the csv contents we generated above. Specific datatypes are enforced to prevent Pandas from dropping leading zeroes, for example, and to make additional operations more streamlined. Each will be explained line by line.

In [ ]:
# generate the file path
gsod_path = os.path.join('.', 'data', 'gsod')

# get the list of yearly weather files
ex_path = os.path.join('.', 'data', 'gsod_extracted')
names = [f for f in os.listdir(ex_path) if 'csv' in f]

# process each year at a time
pbar = tqdm(total=len(names))
for name in names:
    pbar.set_description(name)
    
    # load the data, setting data types explicitly or pandas will drop 
    # the leading zeroes needed for station names. Also, include the 
    # explicit na values designated in the data documentation
    # drop columns we aren't going to use
    f1 = pd.read_csv(
            os.path.join(ex_path, name), 
            dtype={
                'stn' :'str', 
                'wban':'str',
                'moda':'str',
                'frshtt':'str',
                'year':'str'},
            na_values={
                'stn'  :'999999',
                'wban' :'99999',
                'temp' :'9999.9',
                'dewp' :'9999.9',
                'slp'  :'9999.9',
                'stp'  :'9999.9',
                'visib':'999.9',
                'wdsp' :'999.9',
                'mxspd':'999.9',
                'gust' :'999.9',
                'max_temp':'9999.9',
                'min_temp':'9999.9',
                'prcp':'99.9',        
                'sndp':'999.9'},
        ) \
        .drop(columns=[
            'max_temp_flag', 'min_temp_flag', 
            'temp_cnt', 'dewp_cnt', 'slp_cnt', 
            'stp_cnt', 'visib_cnt', 'wdsp_cnt'])

    # convert the two date columns 'year' and 'moda' to a single pydate
    f1['date'] = [
        dt.datetime(year=int(r.year), month=int(r.moda[:2]), day=int(r.moda[2:])) 
        for r in f1.itertuples()
    ]
    
    # extract month number and julian date
    f1['month'] = f1.date.apply(lambda x: x.month)
    f1['doy'] = f1.date.apply(lambda x: x.timetuple().tm_yday)

    # convert prcp values to na where prcp flag is in {'H', 'I'}. see the data docs
    f1.loc[(f1.prcp_flag == 'H') | (f1.prcp_flag == 'I'), 'prcp'] = np.nan

    # convert 'frshtt' to an ordinal value based on severity where the
    # returned value is the number of leading most 1. ie. 010000 -> 2
    # 1:fog, 2:rain, 3:snow, 4:hail, 5:thunderstorm, 6:tornado
    def fx(x):
        x = x[::-1].find('1')
        return x if x != -1 else 0

    f1['atmos_sev'] = f1.frshtt.apply(fx)
    
    # create the join key in the same way as we did for weather stations
    f1.loc[f1.stn.isnull(), 'stn'] = 'none'
    f1.loc[f1.wban.isnull(), 'wban'] = 'none'
    f1['key'] = f1.stn + f1.wban
    
    # perform an inner join with stations
    f1 = f1.merge(stations, on='key', how='inner')
    
    # reorder the columns, dropping the ones that won't be used
    prefix = ['lat', 'lon', 'year', 'month', 'doy']
    f1 = f1.reindex(columns=prefix + sorted(list(
        set(f1.columns) - set(prefix) - {
            'moda', 'prcp_flag', 'frshtt', 'stn', 'wban', 'key', 'date'
        }
    )))
    
    # write the cleaned dataframe to disk
    name = os.path.join(gsod_path, name.replace('.csv', '_cleaned') + '.csv')
    f1.to_csv(name, index=None)
    
    pbar.update(1)
    f1.head()

### Clean the fires dataset
This dataset comes relatively clean. The only modifications we'll be doing is removing the columns we won't be using, creating a few new, and reordering them for convenience.

In [2]:
# generate the path and connect to the sqlite fires file
path = os.path.join('.', 'data', 'FPA_FOD_20170508.sqlite')
conn  = sqlite3.connect(path)

# read all the columns we need
fires = pd.read_sql_query('''
    SELECT FOD_ID,
        FIRE_YEAR, DISCOVERY_DOY, DISCOVERY_TIME,
        STAT_CAUSE_CODE, CONT_DOY, CONT_TIME,
        FIRE_SIZE, LATITUDE, LONGITUDE, OWNER_CODE
    FROM
        Fires;
''', conn)

# convert column names to lowercase
fires.columns = [c.lower() for c in fires.columns]

# based on the first 10000 rows, 0.35% have missing containment values which is a 
# negligible loss at this point in the analysis
fires = fires.dropna(subset=[
    'discovery_doy', 'discovery_time', 'cont_doy', 'cont_time'
], how='any')

# convert fire_year, discovery doy, and time to pydate
fires['dt_disc'] = [
    dt.datetime(year=int(r.fire_year), 
                month=1, 
                day=1, 
                hour=int(r.discovery_time[:2]),
                minute=int(r.discovery_time[2:])
               ) + \
    dt.timedelta(days=r.discovery_doy)
    for r in fires.itertuples()
]

# convert the containment dates
fires['dt_cont'] = [
    dt.datetime(year=int(r.fire_year), month=1, day=1, hour=int(r.cont_time[:2]), minute=int(r.cont_time[2:])) + \
    dt.timedelta(days=r.cont_doy)
    for r in fires.itertuples()
]

# create some higher resolution columns
def seconds_into_year(x):
    a = dt.datetime(year=x.year, month=1, day=1, hour=0, minute=0, second=0)
    return int((x-a).total_seconds())

def seconds_into_day(x):
    a = dt.datetime(year=x.year, month=x.month, day=x.day, hour=0, minute=0, second=0)
    return (x-a).seconds

# calculate fire duration in seconds, but only if the contained date is
# later than the start date
fires['disc_soy'] = fires.dt_disc.progress_apply(seconds_into_year)
fires['cont_soy'] = fires.dt_cont.progress_apply(seconds_into_year)
fires['duration'] = [
    r.cont_soy-r.disc_soy 
    if r.cont_soy > r.disc_soy else np.nan
    for r in tqdm(fires.itertuples(), total=len(fires))
]

# extract month and hour as new columns
fires['date']  = fires.dt_disc.progress_apply(lambda x: x.date())
fires['month'] = fires.dt_disc.progress_apply(lambda x: x.month)
fires['hod']   = fires.dt_disc.progress_apply(lambda x: x.hour)
fires['sod']   = fires.dt_disc.progress_apply(seconds_into_day)

# drop some columns we won't be using
fires = fires.drop(columns=[
    'discovery_time', 'cont_doy', 'cont_time', 
    'disc_soy', 'cont_soy', 'dt_cont', 
    'dt_disc'
])

# rename some columns
fires = fires.rename(columns={
    'discovery_doy':'doy',
    'latitude':'lat',
    'longitude':'lon',
    'fire_year':'year',
    'stat_cause_code':'cause_code',
})

# reorder the columns
prefix = ['fod_id', 'lat', 'lon', 'date', 'year', 'month', 'doy', 'hod', 'sod']
fires = fires.reindex(columns=prefix + sorted(list(
    set(fires.columns) - set(prefix)
)))

100%|██████████| 892007/892007 [00:16<00:00, 55042.22it/s]


Lets take a quick look at the categorical variable - OWNER_CODE.

In [15]:
# generate the path and connect to the sqlite fires file
path = os.path.join('.', 'data', 'FPA_FOD_20170508.sqlite')
conn  = sqlite3.connect(path)

# get the mapping of cause codes to description
owners = pd.read_sql_query('''
    SELECT DISTINCT(OWNER_CODE), OWNER_DESCR
    FROM Fires;
''', conn)\
    .sort_values('OWNER_CODE')

# rename the columns and set the index to code
owners = owners.rename(columns={
    'OWNER_CODE':'code',
    'OWNER_DESCR':'owner'
}).set_index('code')

# get the counts of each cause
bincounts = fires.owner_code.value_counts()

# plot as a bar plot
url = py.plot(go.Figure(
    [go.Bar(
        x=[owners.loc[idx].owner for idx in bincounts.index],
        y=bincounts,
        text=bincounts.index,
        textposition='outside'
    )],
    go.Layout(
        title='Distribution of owners',
        yaxis=dict(title='Count of owned fires')
    )
), filename='wildfires_original_owner_dist')

IFrame(url, width=800, height=600)

This isn't our target variable but there are clear commonalities we can take advantage of to boost any signal that may come from the responsible source. To help understand this a bit better here is the list of federal acronyms:

* USFS - United States Forest Service
* BIA - Bureau of Indian Affairs
* BLM - Bureau of Land Management
* NPS - National Park Service
* FWS - Fish and Wildlife Service
* BOR - Bureau of Reclamation

Here is a list of things I notice from the visualization.
1. UNDEFINED FEDERAL has very little values and can be combined with OTHER FEDERAL. 
2. COUNTY owned land can be joined with MUNICIPAL/LOCAL.
3. STATE OR PRIVATE can be separted into the STATE and PRIVATE categories. To do this, I'll draw from a random binomial distribution characterized by the ratio between the two.
4. TRIBAL can be combined with BIA and I'll rename it to Native American.
5. BOR will have almost no signal on its own so I'll combine it with OTHER FEDERAL.
6. I'll move the FOREIGN items into MISSING/NOT SPECIFIED

I also plan on renaming a few before continuing. Additionally, we'll need to include the new owner descriptions in the cleaned fires dataset so we preserve the recategorization mapping.

In [ ]:
fires.to_csv(os.path.join('.', 'data', 'fires_cleaned.csv'), index=None)
fires.head()

Create a single data frame with cleaned values for all years. This generates a dataframe approximately 1.7gb uncompressed which is a significant reduction from the 3.4gb original compressed file.

In [ ]:
# get the list of cleaned files
files = [f for f in os.listdir(gsod_path) if 'cleaned.csv' in f]
assert len(files) == 25

gsod = pd.concat([pd.read_csv(os.path.join(gsod_path, f)) for f in files])
gsod.to_csv(os.path.join('.', 'data', 'gsod.csv'), index=None)

### Process ORNL features
Each station has a center point and provides the coverage data in both 1km and 50km pixel grids surrounding the station. My first approach to joining the fires and ground cover data was to include any predictions within the station's bounding box but, this led to incredibly sparse results. I leave the cell blocks here to both show my process and why I'm no longer using the data source. In the following cell I load both high and low quality datasets.

In [ ]:
# load the data we'll use, enforce datatypes, and rename columns
cover = pd.concat([
    pd.read_csv(
        os.path.join('.', 'data', f),
        usecols=[
            'LAT_DD', 'LONG_DD', 'COVR1KM', 'COVR50KM'
        ],
        dtype={
            'COVR1KM':'str',
            'COVR50KM':'str'
        }
    ).rename(columns={
        'LAT_DD':'LAT',
        'LONG_DD':'LON'
    })
    for f in [
        'EMDI_ClassA_Cover_UMD_81.csv',
        'EMDI_ClassB_Cover_UMD_933.csv'
    ]
], sort=False)
    
# convert columns to lowercase
cover.columns = [c.lower() for c in cover.columns]

# create cover 50k grid boundaries
cover['lower50_lat'] = cover.lat.apply(lambda x: x-.5)
cover['upper50_lat'] = cover.lat.apply(lambda x: x+.5)
cover['lower50_lon'] = cover.lon.apply(lambda x: x-.5)
cover['upper50_lon'] = cover.lon.apply(lambda x: x+.5)

# only include the values within the fire bounding box
cover = cover.loc[
    (cover.lower50_lat >= min_lat) & (cover.upper50_lat <= max_lat) &
    (cover.lower50_lon >= min_lon) & (cover.upper50_lon <= max_lon)
]

cover.head()

Plot a sample of fires and the bounding boxes for each station to show just how inadequate the ORNL dataset is. Each point represents a fire with the size of the fire mapped to the size of the point.

In [ ]:
# extract a uniform sample of 2k fires
sample = fires.sample(2000)

# generate scatter plot points
fire_trace = go.Scatter(
    x=sample.lon, 
    y=sample.lat,
    mode='markers',
    marker=dict(
        size=np.abs(np.log(sample.fire_size))*3,
        color='#571C00'
    )
)

# generate the bounding boxes
shapes = [
    {
        'type':'rect',
        'x0':r.lower50_lon,
        'x1':r.upper50_lon,
        'y0':r.lower50_lat,
        'y1':r.upper50_lat,
        'fillcolor':'rgba(22, 74, 40, .4)',
        'line':{
            'width':.1
        }
    }
    for r in cover.itertuples()
]

# plot
iplot(go.Figure(
    [fire_trace],
    layout=go.Layout(
        shapes=aRectangles+bRectangles,
        xaxis=dict(
            title='longitude',
            range=[-125, -78]
        ),
        yaxis=dict(
            title='latitude',
            range=[25, 58]
        ),
        title='Ground cover data coverage is insufficient',
        width=1200,
        height=800
    )
))

The same goes for soil content because the same stations are used for this dataset.

In [ ]:
# load the data
soil = pd.concat([
    pd.read_csv(
        os.path.join('.', 'data', f)
    ).rename(columns={
        'LAT_DD':'LAT',
        'LONG_DD':'LON'
    }).drop(columns='SITE_ID')
    for f in [
        'EMDI_ClassA_Soil_IGBP_81.csv',
        'EMDI_ClassB_Soil_IGBP_933.csv'
    ]
], sort=False)

# convert columns to lowercase
soil.columns = [c.lower() for c in soil.columns]

# create the station bounding box
soil['lower50_lat'] = soil.lat.apply(lambda x: x-.5)
soil['upper50_lat'] = soil.lat.apply(lambda x: x+.5)
soil['lower50_lon'] = soil.lon.apply(lambda x: x-.5)
soil['upper50_lon'] = soil.lon.apply(lambda x: x+.5)

# only include the values within the fire bounding box
soil = soil.loc[
    (soil.lower50_lat >= min_lat) & (soil.upper50_lat <= max_lat) &
    (soil.lower50_lon >= min_lon) & (soil.upper50_lon <= max_lon)
]

soil.head()

In [ ]:
# extract a fire sample
sample = fires.sample(2000)

# generate the fire scatter points
fire_trace = go.Scatter(
    x=sample.lon, 
    y=sample.lat,
    mode='markers',
    marker=dict(
        size=np.abs(np.log(sample.fire_size))*3,
        color='#571C00'
    )
)

shapes = [
    {
        'type':'rect',
        'x0':r.lower50_lon,
        'x1':r.upper50_lon,
        'y0':r.lower50_lat,
        'y1':r.upper50_lat,
        'fillcolor':'rgba(22, 74, 40, .4)',
        'line':{
            'width':.1
        }
    }
    for r in soil.itertuples()
]

# plot
iplot(go.Figure(
    [fire_trace],
    layout=go.Layout(
        shapes=aRectangles+bRectangles,
        xaxis=dict(
            title='longitude',
            range=[-125, -78]
        ),
        yaxis=dict(
            title='latitude',
            range=[25, 58]
        ),
        title='Soil data coverage is insufficient',
        width=1200,
        height=800
    )
))

An alternative data source for land coverage is available for public use. See the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/)

### Generate aggregate weather features associated with each fire
We'll need to lookup all reports within a given bounding box centered at the fire's originating location. I use a bounding box to preclude performing pairwise distance lookups which might be more accurate but will incur a significant expense - $O(n^2)$. The embedded hierarchical structure within a degree-decimal formatted coordinate allows us to generate contextually important containment boundaries. The boundaries will include aggregated values from all weather reports $\pm$ 55.5km of the fire.

This is the long running computation may take several days to complete. I wrote it to perform aggregations in batches. Each batch will cache the resulting features to a csv file and continue with the next. Also of note here is that I use a thread pool rather than a process pool to keep memory usage as low as possible.

In [ ]:
# load cleaned GSOD file
gsod = pd.read_csv(os.path.join('.', 'data', 'gsod.csv'))
gsod.head()

In [ ]:
# start a thread pool and progress bar
pool = TPool(mul.cpu_count())
pbar = tqdm(total=len(fires))


def weather_agg(args):    
    try:
        # extract the tuple arguments
        fod_id, lat, lon, year, doy = args
        
        # make a copy of the empty record to start this record with
        results = empty.copy()
        results['fod_id'] = fod_id
        
        # get all weather reports within 111km
        lat_min, lat_max = lat-.5, lat+.5
        lon_min, lon_max = lon-.5, lon+.5
        
        # retrieve all weather reports within the box and 4 days leading up to and including
        # the day of the fire
        wthr = gsod.loc[
            (gsod.lat >= lat_min) & (gsod.lat <= lat_max) &
            (gsod.lon >= lon_min) & (gsod.lon <= lon_max) &
            (
                (gsod.year == year) & (gsod.doy >= doy-4) & (gsod.doy <= doy) |
                (gsod.doy <= 4) & (gsod.year == year-1) & (gsod.doy >= 361+doy)
            )
        ]
        
        # get the three day prior aggregates
        w_ = wthr.loc[wthr.doy != doy]
        if len(w_) > 0:
            results['threeDay_atmos_sev'] = np.mean(w_.atmos_sev)
            results['threeDay_temp_max']  = np.max(w_.temp_max)
            results['threeDay_temp_min']  = np.min(w_.temp_min)
            results['threeDay_temp']  = np.median(w_.temp)
            results['threeDay_sndp']  = np.median(w_.sndp)
            results['threeDay_dewp']  = np.median(w_.dewp)
            results['threeDay_gust']  = np.max(w_.gust)
            results['threeDay_mxspd'] = np.max(w_.mxspd)
            results['threeDay_stp']   = np.median(w_.stp)
            results['threeDay_temp']  = np.median(w_.temp)
            results['threeDay_slp']   = np.median(w_.slp)
            results['threeDay_wdsp']  = np.median(w_.wdsp)
            results['threeDay_prcp']  = np.sum(w_.prcp)
            results['threeDay_visib'] = np.median(w_.visib)   

        # get the dayOf aggregates   
        w_ = wthr.loc[wthr.doy == doy]
        if len(w_) > 0:            
            results['dayOf_atmos_sev'] = np.mean(w_.atmos_sev)
            results['dayOf_temp_max']  = np.max(w_.temp_max)
            results['dayOf_temp_min']  = np.min(w_.temp_min)
            results['dayOf_temp']  = np.median(w_.temp)
            results['dayOf_sndp']  = np.median(w_.sndp)
            results['dayOf_dewp']  = np.median(w_.dewp)
            results['dayOf_gust']  = np.max(w_.gust)
            results['dayOf_mxspd'] = np.max(w_.mxspd)
            results['dayOf_stp']   = np.median(w_.stp)
            results['dayOf_temp']  = np.median(w_.temp)
            results['dayOf_slp']   = np.median(w_.slp)
            results['dayOf_wdsp']  = np.median(w_.wdsp)
            results['dayOf_prcp']  = np.median(w_.prcp)
            results['dayOf_visib'] = np.median(w_.visib)   
    
    # catch all exceptions and continue gracefully but make sure we
    # notify in case any occur
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(exc_type, e, exc_tb.tb_lineno)
    
    pbar.update(1)
    return results

# create the dayOf columns
excols = {'lat', 'lon', 'elev', 'year', 'month', 'doy', 'fod_id'}
daily_cols    = ['dayOf_'    + c for c in list(set(gsod.columns) - excols)]
threeDay_cols = ['threeDay_' + c for c in list(set(gsod.columns) - excols)]

# create an empty dictionary to start each feature row
empty = dict()
for c in daily_cols+threeDay_cols:
    empty[c] = np.nan

# perform this operation in batches caching the fire results each iteration
start, step = 0, 10000
for i in range(0, len(fires), step):
    # get the set of indices to process
    idx_set = fires.index.tolist()[i:i+step]
    
    # process
    batch = pool.map(weather_agg, [
        (r.fod_id, r.lat, r.lon, r.year, r.doy) 
        for r in fires.loc[idx_set].itertuples()
    ])
    
    # cache
    pd.DataFrame(batch).to_csv(os.path.join('.', 'data', 'fires', f'fires_b{i}.csv'), index=None)
    
pool.close(); pool.join()

Finally, read all batches into a single dataframe and write it back to disk as one.

In [ ]:
# combine the batches into a single dataframe
path = os.path.join('.', 'data', 'fires')
fire_weather = pd.concat(
    [
        pd.read_csv(os.path.join(path, f)) 
        for f in os.listdir(path) if '.csv' in f
    ],
    sort=False
)

# write the combined dataframe to disk
path = os.path.join('.', 'data', 'fire_weather.csv')
fire_weather.to_csv(path, index=None)

fire_weather.head()

### Create the combined file to use for analysis

In [ ]:
# load the cleaned fires data
path = os.path.join('.', 'data', 'fires_cleaned.csv')
fires = pd.read_csv(path, parse_dates=['date'])

# load the weather aggregations
path = os.path.join('.', 'data', 'fire_weather.csv')
weather = pd.read_csv(path)

# merge the dataframes on the fod_id
df = fires.merge(weather, on='fod_id')

In [ ]:
def nan_percentages(df, show_zero=False):
    cols = sorted(df.columns)
    d, p = len(df), {}

    for col in cols:
        a = sum(pd.isnull(df[col]))
        p[col] = a/d

    for k, v in p.items():
        n = len(k) if len(k) <= 20 else 20
        v = np.round(v, 4)
        
        if v != 0 or show_zero:
            print('{:<20} {:<5}'.format(k[:n], v))

compute_cols = list(set(df.columns) - {'fod_id', 'date'})        
nan_percentages(df[compute_cols])

First off, we notice that nearly 13% of our rows weren't recorded correctly. Those are the records where the contanment date was recorded before the discovery date. Let's drop those records.

In [ ]:
df = df.loc[[not b for b in df.duration.isnull()]]

We have quite a few NA values in the resulting weather data and I'm running out of time to do any complex fixes. For the purposes of this project we're going to make some quick assumptions and transformations. Lets see how much of the dataset doesn't have any dayOf features at all.

In [ ]:
np.round(len(df.loc[
    df.dayOf_prcp.isnull() &
    df.dayOf_visib.isnull() &
    df.dayOf_gust.isnull() &
    df.dayOf_dewp.isnull() & 
    df.dayOf_temp_max.isnull() &
    df.dayOf_temp_min.isnull() &
    df.dayOf_temp.isnull() &
    df.dayOf_atmos_sev.isnull() &
    df.dayOf_wdsp.isnull() &
    df.dayOf_mxspd.isnull()
])/len(df)*100, 1)

That's quite a high percentage and accounts for many of the missing values. Lets drop those records.

In [ ]:
df = df.dropna(subset=[
    'dayOf_prcp', 'dayOf_visib', 'dayOf_gust', 'dayOf_dewp',
    'dayOf_temp_max', 'dayOf_temp_min', 'dayOf_temp',
    'dayOf_atmos_sev', 'dayOf_wdsp', 'dayOf_mxspd'
], how='all')

Next lets look at sndp - snow depth. This column is almost completely nan but we don't have to lose the information. Lets transform this column into an indicator that simply says whether or not snow was present at all.

In [ ]:
# create the indicators
df['threeDay_snow'] = [1 if not b else 0 for b in df.threeDay_sndp.isnull()]
df['dayOf_snow']    = [1 if not b else 0 for b in df.dayOf_sndp.isnull()]

# drop the original
df = df.drop(columns=['threeDay_sndp', 'dayOf_sndp'])

The next highest source of missing values is in our pressure columns: slp and stp. I'm going to drop these columns all together.

In [ ]:
# drop the pressure columns
df = df.drop(columns=[
    'dayOf_stp', 'dayOf_slp', 'threeDay_stp', 'threeDay_slp'
])

Now lets take the missing gust values. For this, lets just take the maximum recorded windspeed for the day and three day respectively.

In [ ]:
df.loc[df.dayOf_gust.isnull(), 'dayOf_gust'] = df.loc[df.dayOf_gust.isnull(), 'dayOf_mxspd']
df.loc[df.threeDay_gust.isnull(), 'threeDay_gust'] = df.loc[df.threeDay_gust.isnull(), 'threeDay_mxspd']

I use linear regression models to impute any of the remaining missing values. In the next cell, I loop through each collumn with missing values generating a model for each. I use these individual models to predict the remaining missing values. This preserves any existing relationship that may exist between the independent variables.

In [ ]:
# get the remaining columns with nan values
to_impute = [c for c in df.columns if sum(df.loc[:, c].isnull()) > 0]

# make sure we don't use these columns in the regression model
excluded_columns = {
    'fod_id', 'date', 'year', 'sod', 'cause_code', 
    'duration', 'fire_size', 'owner_code', 'dayOf_snow',
    'threeDay_snow'
}

# impute each column
for c in tqdm(to_impute):
    # extract the rows that need imputed
    x = df[[not b for b in df.loc[:, c].isnull()]]
    
    # get the column names to use 
    inputs = set(df.columns) - excluded_columns - {c}
    
    # create the r-style formula
    formula = c + '~' + '+'.join(inputs)
    
    # build and fit the model
    model = smf.ols(formula=formula, data=df).fit()
    
    # make predictions
    predictions = model.predict(exog=df.loc[
        df.loc[:, c].isnull(), inputs
    ])
    
    # ensure predictions aren't negative
    predictions = [p if p > 0 else 0 for p in predictions]
    
    # set the missing vals to the predicted
    df.loc[df.loc[:, c].isnull(), c] = predictions

As a final check lets print the percentage of nan values to make sure we've generated a complete dataset for analysis.

In [ ]:
compute_cols = list(set(df.columns) - {'fod_id', 'date'})    
nan_percentages(df[compute_cols], show_zero=True)

In [ ]:
# write it to disk
path = os.path.join('.', 'data', 'fires_complete.csv')
df.to_csv(path, index=None)

# show it
df.head()

## Part 2. Analysis

### Q1. What are the most important indicators when determining the strength of a wildfire?

First, lets get an idea of the distribution of classes in the dataset. We'll do this by querying the distinct cause descriptions for the original fires dataset and visualizing the counts of each.

In [ ]:
# load the cleaned fire data
path = os.path.join('.', 'data', 'fires_complete.csv')
df = pd.read_csv(path, parse_dates=['date'])

In Part 1 we left the dataframe in a more dense form by not expanding the categorical variables. We only have one of those features: owner code. Lets go ahead and convert them before proceeding any further. First, I query the orginal fires dataset to get the mapping of owner code to description to give us a bit more context. 

In [ ]:
df.columns

In [ ]:
# generate the path and connect to the sqlite fires file
path = os.path.join('.', 'data', 'FPA_FOD_20170508.sqlite')
conn  = sqlite3.connect(path)

# get the mapping of cause codes to description
cause_map = pd.read_sql_query('''
    SELECT DISTINCT(STAT_CAUSE_CODE), STAT_CAUSE_DESCR
    FROM Fires;
''', conn)\
    .sort_values('STAT_CAUSE_CODE')

# rename the columns and set the index to code
cause_map = cause_map.rename(columns={
    'STAT_CAUSE_CODE':'code',
    'STAT_CAUSE_DESCR':'cause'
}).set_index('code')

In [ ]:
# get the counts of each cause
bincounts = df.cause_code.value_counts()

# plot as a bar plot
iplot(go.Figure(
    [go.Bar(
        x=[cause_map.loc[idx].cause for idx in bincounts.index],
        y=bincounts,
    )],
    go.Layout(
        title='Distribution of causes is not uniformly distributed',
        yaxis=dict(title='Count of fires')
    )
))

From this visualization we see difficulties beginning to form. The classes are far from uniformly distributed which makes predicting the lower represented classes more difficult.

In [ ]:
from sklearn.decomposition import PCA

compute_cols

pca = PCA()

In [ ]:
pca.shape